# Libraries

In [2]:
import pandas as pd
import polars as pl
import numpy as np
import gc
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import StratifiedGroupKFold

# Configurations

In [3]:
class CONFIG:
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.09
    start_dt = 1450

In [4]:
#1 698 max date 
# 120 for validation => 1 578 left => 200 for train => 1 378

# Load training data

In [5]:
# Use last 2 parquets
train = pl.scan_parquet(
    f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)*2).cast(pl.Int32).alias("label"),
).filter(
    pl.col("date_id").gt(CONFIG.start_dt)
)

# Create Lags data from training data

In [6]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
    )
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
lags

# Merge training data and lags data

In [7]:
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")

# Split training data and validation data

In [10]:
len_train   = train.select(pl.col("date_id")).collect().shape[0]
valid_records = int(len_train * CONFIG.valid_ratio)
len_ofl_mdl = len_train - valid_records
last_tr_dt  = train.select(pl.col("date_id")).collect().row(len_ofl_mdl)[0]

print(f"\n len_train = {len_train}")
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
print(f"\n---> Last offline train date = {last_tr_dt}\n")

training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data   = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 9144696

 len_ofl_mdl = 8321674

---> Last offline train date = 1677



In [9]:
# validation_data

# Save data as parquets

In [11]:
training_data.collect().\
write_parquet(
    f"training.parquet", partition_by = "date_id",
)

In [12]:
validation_data.collect().\
write_parquet(
    "validation.parquet", partition_by = "date_id",
)

## Catboost

In [11]:
# Custom R2 metric for CatBoost
class r2_cbt(object):
    def get_final_error(self, error, weight):
        return 1 - error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w * (target[i] ** 2)
            error_sum += w * ((approx[i] - target[i]) ** 2)

        return error_sum, weight_sum

In [12]:
X_train = train.select([f"feature_{idx:02d}" for idx in range(79)] + ['responder_6_lag_1', 'symbol_id']).collect().to_numpy().astype('float32')
y_train = train.select('responder_6').collect().to_numpy().astype('float32')
weights = train.select('weight').collect().to_numpy().astype('float32').flatten()

In [13]:
import joblib
import catboost as cbt

model = cbt.CatBoostRegressor(iterations=1000,
                              learning_rate=0.05, 
                         #     task_type='GPU', 
                              loss_function='RMSE', 
                              eval_metric=r2_cbt())


# evalset = cbt.Pool(X_val, y_val, weight=w_val)

# del X_val, y_val
# gc.collect()

# Train CatBoost model with early stopping and verbose logging
model.fit(X_train, y_train, sample_weight=weights, 
         
          # eval_set=[evalset], 
          verbose=10, 
          early_stopping_rounds=100)

joblib.dump(model, 'model_catboost_lag_32_18.pkl')


0:	learn: 0.0006330	total: 2.92s	remaining: 48m 39s
10:	learn: 0.0045487	total: 18.6s	remaining: 27m 51s
20:	learn: 0.0068677	total: 31.7s	remaining: 24m 37s
30:	learn: 0.0085148	total: 46s	remaining: 23m 57s
40:	learn: 0.0102077	total: 1m 2s	remaining: 24m 22s
50:	learn: 0.0116773	total: 1m 17s	remaining: 24m 4s
60:	learn: 0.0135298	total: 1m 31s	remaining: 23m 35s
70:	learn: 0.0150504	total: 1m 45s	remaining: 23m 6s
80:	learn: 0.0167032	total: 2m	remaining: 22m 43s
90:	learn: 0.0181374	total: 2m 15s	remaining: 22m 31s
100:	learn: 0.0197942	total: 2m 29s	remaining: 22m 7s
110:	learn: 0.0210923	total: 2m 43s	remaining: 21m 48s
120:	learn: 0.0222168	total: 2m 59s	remaining: 21m 41s
130:	learn: 0.0229690	total: 3m 13s	remaining: 21m 21s
140:	learn: 0.0245182	total: 3m 29s	remaining: 21m 16s
150:	learn: 0.0255708	total: 3m 42s	remaining: 20m 53s
160:	learn: 0.0261965	total: 3m 57s	remaining: 20m 37s
170:	learn: 0.0268051	total: 4m 10s	remaining: 20m 16s
180:	learn: 0.0277587	total: 4m 24s

['model_catboost_lag_32_18.pkl']

In [14]:
# train_set = train.select(pl.col('responder_6')).collect().to_pandas()
# train_set= train_set['responder_6'].to_list()

In [15]:
# # Geometric Brownian Motion (GBM)

# # Parameter Definitions

# # So    :   initial stock price
# # dt    :   time increment -> a day in our case
# # T     :   length of the prediction time horizon(how many time points to predict, same unit with dt(days))
# # N     :   number of time points in prediction the time horizon -> T/dt
# # t     :   array for time points in the prediction time horizon [1, 2, 3, .. , N]
# # mu    :   mean of historical daily returns
# # sigma :   standard deviation of historical daily returns
# # b     :   array for brownian increments
# # W     :   array for brownian path


# # Parameter Assignments
# scen_size = 1000
# So = train_set[-1]
# dt = 1  # time id  
# # n_of_wkdays = pd.date_range(start=pd.to_datetime(end_date,
# #                                                  format="%Y-%m-%d") + pd.Timedelta('1 days'),
# #                             end=pd.to_datetime(pred_end_date,
# #                                                format="%Y-%m-%d")).to_series().map(lambda x: 1 if x.isoweekday() in range(1, 6) else 0).sum()

# T = len(train_set)
# N = T / dt
# t = np.arange(1, int(N) + 1)
# mu = np.mean(train_set)
# sigma = np.std(train_set)
# b = {str(scen): np.random.normal(0, 1, int(N)) for scen in range(1, scen_size + 1)}
# W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}


# # Calculating drift and diffusion components
# drift = (mu - 0.5 * sigma ** 2) * t
# diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

In [13]:
# # Making the predictions
# S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
# S = np.hstack((np.array([[So] for scen in range(scen_size)]), S))  # add So to the beginning series
# S_max = [S[:, i].max() for i in range(0, int(N))]
# S_min = [S[:, i].min() for i in range(0, int(N))]
# S_pred = .5 * np.array(S_max) + .5 * np.array(S_min)
# final_df = pd.DataFrame(data=[test_set.reset_index()['Adj Close'], S_pred],
#                         index=['real', 'pred']).T
# final_df.index = test_set.index
# mse = 1/len(final_df) * np.sum((final_df['pred'] - final_df['real']) ** 2)

# # Plotting the simulations
# plt.rcParams["font.family"] = "serif"
# fig, ax = plt.subplots()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.suptitle('Monte-Carlo Simulation: ' + str(scen_size) + ' simulations', fontsize=20)
# plt.title('Asset considered: {}'.format(stock_name))
# plt.ylabel('USD Price')
# plt.xlabel('Prediction Days')
# for i in range(scen_size):
#     plt.plot(pd.date_range(start=train_set.index[-1],
#                            end=pred_end_date,
#                            freq='D').map(lambda x: x if x.isoweekday() in range(1, 6) else np.nan).dropna(), S[i, :])
# plt.show()

# # Plotting the final prediction against the real price
# fig, ax = plt.subplots()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.suptitle('Predicted Price vs Real Price', fontsize=20)
# plt.title('Mean Squared Error (MSE): {}'.format(np.round(mse, 2)))
# plt.ylabel('USD Price')
# plt.plot(final_df)
# plt.legend(['Real Price', 'Predicted Price'],
#            loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2, frameon=False)
# plt.show()